# Experiment: learning graph filters

1. Random kNN graph: $W \in \mathbb{R}^{M \times M} \rightarrow L \rightarrow U, \Lambda$.
2. Random graph signals: $X = \{x_i\}_{i=1}^N \in \mathbb{R}^{M \times N}$.
3. Linear mapping: $f(x_i, c) = U \operatorname{diag}(c) U^T x_i$.
4. Noisy target signals: $Y = \{y_i\}_{i=1}^N \in \mathbb{R}^{M \times N}, y_i = f(x_i, c_{gt}) + \mathcal{N}_M(0,\epsilon)$.
    1. With randomly generated coefficients $c_{gt} \sim \mathcal{N}_M(0,1)$.
5. Convex and smooth loss function: $L = \frac{1}{N} \sum_{i=1}^N \|f(x_i, c) - y_i\|_2^2 = \frac{1}{N} \|U \operatorname{diag}(c) U^TX - Y\|_F^2$.
    1. Gradient: $\nabla_{c} L = \frac{2}{N} \left(U^T X \circ ( c \circ U^T X - U^T Y ) \right) 1_N$.
6. Optimization: $c^* = \operatorname{arg min}_c L(c)$.
7. Verification.
    1. $c^*$ should converge to $c_{gt}$.
    2. The loss $L(c^*)$ should converge to $L(c_{gt})$.

In [ ]:
import random, time
import numpy as np
import scipy.sparse, scipy.sparse.linalg, scipy.spatial.distance
import matplotlib.pyplot as plt
%matplotlib inline
tol = 1e-10

## Setting

### Graph

* A completely random graph is not smooth at all and will thus have a large spectral gap, i.e. $\lambda_1 >> \lambda_0$.
* A grid, on the contrary, is very regular.

In [ ]:
M = 100  # nodes
k = 4  # edges per vertex

def graph_random():
    """Random connections and weights."""
    I = np.arange(0, M).repeat(k)
    J = np.random.randint(0, M, M*k)
    V = np.random.uniform(0, 1, M*k)
    return scipy.sparse.coo_matrix((V, (I, J)), shape=(M, M))

def graph_grid():
    """Construct a kNN graph aranged on a 2D grid."""
    
    # Construct a grid.
    m = np.int(np.sqrt(M))
    x = np.linspace(0,1,m)
    y = np.linspace(0,1,m)
    xx, yy = np.meshgrid(x, y)
    z = np.empty((M,2))
    z[:,0] = xx.reshape(M)
    z[:,1] = yy.reshape(M)

    # Compute pairwise distances.
    d = scipy.spatial.distance.pdist(z, 'euclidean')
    d = scipy.spatial.distance.squareform(d)

    # k-NN graph.
    idx = np.argsort(d)[:,1:k+1]
    d.sort()
    d = d[:,1:k+1]

    # Weights.
    sigma2 = np.mean(d[:,-1])**2
    d = np.exp(- d**2 / sigma2)

    # Weight matrix.
    I = np.arange(0, M).repeat(k)
    J = idx.reshape(M*k)
    V = d.reshape(M*k)
    return scipy.sparse.coo_matrix((V, (I, J)), shape=(M, M))

W = graph_grid()

# No self-connections.
W.setdiag(0)

# Non-directed graph.
bigger = W.T > W
W = W - W.multiply(bigger) + W.T.multiply(bigger)
del bigger
assert np.abs(W - W.T).mean() < tol

# CSR sparse matrix format for efficient multiplications.
W = W.tocsr()
W.eliminate_zeros()

print("{} > {} edges".format(W.nnz, M*k))

* $L^\text{unnormalized} = D - W$
* $L^\text{normalized} = I - D^{-1/2} W D^{-1/2}$

In [ ]:
normalized_laplacian = True

def laplacian(W, normalized=True):
    """Return the Laplacian of the weigth matrix."""
    
    # Degree matrix.
    d = W.sum(axis=0)

    # Laplacian matrix.
    if not normalized:
        D = scipy.sparse.diags(d.A.squeeze(), 0)
        return D - W
    else:
        d = 1 / np.sqrt(d)
        D = scipy.sparse.diags(d.A.squeeze(), 0)
        I = scipy.sparse.identity(M, dtype=D.dtype)
        return I - D * W * D


t_start = time.process_time()
LL = laplacian(W, normalized_laplacian)
print('Execution time: {:1.0f}ms'.format((time.process_time() - t_start) * 1000))
assert np.abs(LL - LL.T).mean() < tol

$L = U^T \Lambda U$ where $\Lambda$ is a diagonal matrix of eigenvalues.
Compare the results of four algorithms.

In [ ]:
def fourier(L):

    def sort(lamb, U):
        idx = lamb.argsort()
        return lamb[idx], U[:,idx]

    t_start = time.process_time()
    lamb, U = np.linalg.eig(LL.toarray())
    lamb, U = sort(lamb, U)
    print('Execution time: {:1.0f}ms'.format((time.process_time() - t_start) * 1000))

    if M <= 100:  # Because of the computational complexity.
        
        lamb_, U_ = np.linalg.eigh(LL.toarray())
        np.testing.assert_allclose(lamb_, lamb, atol=tol)
        np.testing.assert_allclose(np.abs(U_), np.abs(U), atol=tol)

        lamb_, U_ = scipy.sparse.linalg.eigs(LL, k=M-2, which='SM')
        lamb_, U_ = sort(lamb_, U_)
        np.testing.assert_allclose(lamb[:-2], lamb_, atol=tol)
        np.testing.assert_allclose(np.abs(U[:,:-2]), np.abs(U_), atol=tol)

        lamb_, U_ = scipy.sparse.linalg.eigsh(LL, k=M-1, which='SM')
        np.testing.assert_allclose(lamb[:-1], lamb_, atol=tol)
        np.testing.assert_allclose(np.abs(U[:,:-1]), np.abs(U_), atol=tol)
    
    return lamb, U

lamb, U = fourier(LL)

Upper-bound approximation of the spectrum.

* Computed by the Implicitly Restarted Lanczos Method (IRLM), which is a reduction of a variant of the Arnoldi iteration. It is faster than the Power method.
* Normalized graph Laplacian has a bounded spectrum $0 \leq \lambda \leq 2$.
* `eigs` is faster than `eigsh`. There are also non-sparse routines in `scipy.linalg`.

In [ ]:
t_start = time.process_time()
lmax = scipy.sparse.linalg.eigsh(LL, k=1, which='LM', return_eigenvectors=False)[0]
print('Execution time: {:1.0f}ms'.format((time.process_time() - t_start) * 1000))
if normalized_laplacian:
    assert lmax <= 2
print('Spectrum: [{:1.2e}, {:1.2e}]'.format(lamb[0], lmax))
np.testing.assert_allclose(lamb[0], 0, atol=tol)
np.testing.assert_allclose(lamb[-1], lmax, atol=tol)

### Ground truth graph filter

Linear mapping $f(x_i, c) = U C U^T x_i$, $C$ is the diagonal matrix $C = \operatorname{diag}(c)$, i.e. $c = C 1_M$.

* Parametrized low-pass filter coefficients $(c_{gt})_i = \operatorname{e}^{-t \lambda_i}$
* Random filter coefficients $c_{gt} \sim \mathcal{N}_M(0,1)$

In [ ]:
parametrized = True

if parametrized:
    def g(x, t=.5):
        return np.sin(4 * (x-2)**2)
        return np.sin(2 * (x-2)**2)
        return np.exp(-t * x)
    c_g = g(lamb)
    #assert np.all(c_gt[:-1] - c_gt[1:] > 0)
else:
    c_g = np.random.normal(0, 1, M)

### Signals

* Random input signals $X \sim \mathcal{N}_{M \times N}(0,1)$
  * Low-pass signals ?
* Noisy target signals $y_i = f(x_i, c_{gt}) + \mathcal{N}_M(0,\epsilon)$

In [ ]:
N = 200  # signals
eps = 0.1  # noise

X = np.random.normal(0, 1, (M,N))
Y = U @ np.diag(c_g) @ U.T @ X + (np.random.normal(0, eps, (M,N)) if eps > 0 else 0)

## Non-parametrized filter learning

### Loss function

* Loss function $L = \frac{1}{N} \sum_{i=1}^N \|f(x_i, c) - y_i\|_2^2 = \frac{1}{N} \|UCU^TX - Y\|_F^2$.
    * Spectral domain: $L = \frac{1}{N} \| C U^T X - U^T Y \|_F^2$.
    * Independant coefficients: $L = \frac{1}{N} \sum_{i=1}^M \| c_i (U^T X)_{i,\cdot} - (U^T Y)_{i,\cdot} \|_2^2$.
    * Convex and smooth w.r.t. $c$.
* Gradient:
    * Independant coefficients: $\nabla_{c_i} L = \frac{2}{N} ( c_i (U^T X)_{i,\cdot} - (U^T Y)_{i,\cdot} ) (X^T U)_{\cdot,i}$.
    * $\nabla_{c} L = \frac{2}{N} \left(U^T X \circ ( c \circ U^T X - U^T Y ) \right) 1_N$.
* Optimization $c^* = \operatorname{arg min}_{c} L(c)$

In [ ]:
def filter_full(X, c):
    """Filter X with a full spectral domain filter."""
    return U @ np.diag(c) @ U.T @ X
np.testing.assert_allclose(filter_full(X, np.ones(M)), X, atol=tol)

def L(c):
    M, N = X.shape
    return np.linalg.norm(filter_full(X, c) - Y, ord='fro')**2 / N
np.testing.assert_allclose(L(c_g), M * eps**2, 2e-2)

def dL(X, Y, c, variant=None):
    M, N = X.shape
    A = U.T @ X
    B = U.T @ Y
    # Speed: v3 >> v1 > v2.
    if variant is 1:
        return 2 / N * np.diag(A @ (A.T @ np.diag(c) - B.T))
    elif variant is 2:
        dc = np.empty(M)
        for i in range(M):
            dc[i] = 2 / N * (c[i] * A[i,:] - B[i,:]) @ A.T[:,i]
        return dc
    else:
        # Speed: .sum(axis=1) is faster than *np.ones(N).y
        return 2 / N * (A * (c[:,np.newaxis] * A - B)).sum(axis=1)
# Gradient should be null at the global minimum. With noise, c_g is not necessary the optimum.
if eps <= 0:
    np.testing.assert_allclose(dL(X, Y, c_g), 0, atol=tol)
np.testing.assert_allclose(dL(X, Y, c_g), dL(X, Y, c_g, 1))
np.testing.assert_allclose(dL(X, Y, c_g), dL(X, Y, c_g, 2))

### Optimization: optimality condition

* Only possible because $L$ is convex and smooth.
* Optimality condition $\nabla_c L = 0$ gives $(U^T X \circ U^T X) 1_N \circ c = (U^T X \circ U^T Y) 1_N$.

In [ ]:
t_start = time.process_time()
A = U.T @ X
B = U.T @ Y
c_o = (A * B).sum(axis=1) / (A * A).sum(axis=1)
print('Execution time: {:1.0f}ms'.format((time.process_time() - t_start) * 1000))

assert L(c_o) < L(c_g) + tol
assert np.linalg.norm(dL(X, Y, c_o)) < np.linalg.norm(dL(X, Y, c_g))
np.testing.assert_allclose(dL(X, Y, c_o), 0, atol=tol)
if eps <= 0:
    np.testing.assert_allclose(c_o, c_g, atol=tol)
    np.testing.assert_allclose(L(c_o), L(c_g), atol=tol)

### Optimization: stochastic (mini-batch) gradient descent

* Works also for $L$ which are non-smooth (with sub-gradient) or non-convex.
* Idea: descend the gradient of the loss function.
* Efficiency: compute the gradient $\nabla_c L$ with a sub-set (mini-batch) of the training data.
    * Extreme case: one sample at a time. Very inefficient.
* Update rule (gradient descent) $c^{n+1} = c^n - \lambda_n \nabla_c L$.
* Note: objective (loss on training set) and error (on validation set) are usually evaluated after each epoch. The algorithm is thus stopped after a maximum number of epochs rather than iterations.
* Hyper-parameters.
    * Learning rate (step size) $\lambda_n$. Bigger the batch size, smaller the learning rate.
        * Tradeoff.
            * Small: progress is steady but slow.
            * Big: risks of oscillations or divergence.
        * There are tricks, e.g. vanishing step (like simulated annealing).
    * Size of the mini-batch.
        * We want the one who minimizes the *training time*.
        * Trade-off: should be limited by the available memory, somewhere around 100.
            * Larger is more stable, but computationnaly more expensive.
            * Smaller demands more accesses to memory, which is slow.
            * Larger exploits the parallelism of modern hardware architectures (SIMD on CPU, GPU).
        * Extreme cases:
            * $1$: stochastic gradient descent.
            * $N$: gradient descent.
    * Stopping criterion.
        * Convergence of the loss function $L$.
        * Convergence of the parameters $c$.
        * Maximum number of iterations.

In [ ]:
def sgd(c0, L, dL, learning_rate=.1, batch_size=100, conv=1e-3, maxit=100, window=10):
    """Stochastic (mini-batch) gradient descent."""
    indices = []
    c = c0
    loss = [L(c)]
    
    def stop(loss):
        """Stop after convergence of the loss."""
        if len(loss) > maxit:
            return True
        #elif np.linalg.norm(dL(X, Y, c)) < conv:
            #return True
        elif len(loss) >= 2 * window:
            avg1 = np.mean(loss[-window:])
            avg2 = np.mean(loss[-2*window:-window])
            return True if avg2 - avg1 < conv else False
        else:
            return False
    
    while not stop(loss):
        
        # Be sure to have used all the samples before using one a second time.
        if len(indices) < batch_size:
            new_indices = np.arange(N)
            np.random.shuffle(new_indices)
            indices.extend(new_indices)
        idx = indices[:batch_size]
        del indices[:batch_size]
        
        c -= learning_rate * dL(X[:,idx], Y[:,idx], c)
        loss.append(L(c))
        
    return c, loss

In [ ]:
def sgd_plot_convergence(c0, L, dL, params, conv, maxit):
    fig = plt.figure(figsize=(15,5))
    ax = fig.add_subplot(1,1,1)

    c_sgd = []
    mlen = 0
    for param in params:
        t_start = time.process_time()
        c, loss = sgd(np.array(c0), L, dL, param[0], param[1], conv, maxit)
        t = (time.process_time() - t_start) * 1000
        label = 'rate={}, size={}, L(c)={:1.2e}, |dL(c)|={:1.2e}, time={:1.0f}ms'.format(
            param[0], param[1], L(c), np.linalg.norm(dL(X, Y, c)), t)
        ax.plot(loss, label=label)
        c_sgd.append(c)
        mlen = max(mlen, len(loss))
        
    ax.set_title('Convergence, M={}, N={}, eps={}'.format(M, N, eps))
    ax.set_xlabel('iteration n')
    ax.set_ylabel('loss L(c^n)')
    ax.legend(loc='best')
    ax.set_xlim(0, mlen-1)
    plt.show()
    
    return c_sgd

params = []
params.append([0.2, 1])
params.append([0.2, 5])
params.append([0.2, 50])
params.append([0.2, 100])
params.append([0.6, 100])
c0 = np.random.uniform(0, 1, M)
c_s = sgd_plot_convergence(c0, L, dL, params, conv=1e-3, maxit=100)

### Results: learned filters

Observations:
* Noise: why don't we find the same loss as the ground truth, but the same as linear programming ?
    * The gradient was incorrectly set to $\nabla_c L = \frac{2}{N} U^T X (X^T U c - Y^T U 1_M)$.
* More samples, e.g. $N=2000$: why don't we find the same loss as the linear program ?
    * Learning rate too high.
* The spectral gap $\lambda_1$ is large for a random graph.
* Without noise, the recovered filter is exact.

In [ ]:
def plot_filters(coeffs):
    fig = plt.figure(figsize=(15,5))
    ax = fig.add_subplot(1,1,1)
    for coeff in coeffs:
        c = eval(coeff)
        label = '{}: L={:1.2e}, |dL|={:1.2e}'.format(coeff, L(c), np.linalg.norm(dL(X,Y,c)))
        ax.plot(lamb, c, '.-', label=label)
    ax.set_xlim(lamb[0], lamb[-1])
    ax.set_title('Filter coefficients, M={}, N={}, eps={}'.format(M, N, eps))
    ax.set_xlabel('frequency')
    ax.set_ylabel('amplitude')
    ax.legend(loc='best')

plot_filters(['c_s[4]', 'c_s[0]', 'c_o', 'c_g'])

## Parametrized filter learning: truncated Chebyshev expansion

* Use a $K$th order polynomial approximation of the filter.
* Less free parameters: $K << M$.
* Good approximation for smooth, i.e. localized, filters.

### Basis of Chebyshev polynomials

* Compute the Chebyshev basis $T$ of order $K$.
* This basis will allow us to construct and observe the filter from the inferred polynomial coefficients.
* The figure shows that we indeed generate the Chebyshev polynomials of the first kind.

In [ ]:
K = 5

def cheby_basis(K, x):
    """Return the Chebyshev basis of order K (composed of the
    first K polynomials) evaluated at x. Polynomials are generated
    by their recursive formulation."""
    T = np.empty((x.size, K))
    T[:,0] = np.ones(x.size)
    if K >= 2:
        T[:,1] = x
    for k in range(2, K):
        T[:,k] = 2 * x * T[:,k-1] - T[:,k-2]
    return T

fig = plt.figure(figsize=(15,5))
ax = fig.add_subplot(1,1,1)
x = np.linspace(-1,1,100)
T = cheby_basis(K, x)
for k in range(K):
    ax.plot(x, T[:,k], label='T_{}'.format(k))
ax.set_title('Chebyshev polynomials of the first kind')
ax.set_xlabel('x')
ax.set_ylabel('T_n(x)')
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1.1)
ax.legend(loc='best')
plt.show()

### Ground truth Chebyshev expansion's coefficients

Given the filter $g$ with a vector $c_{gt} \in \mathbb{R}^M$ of evaluations, find the Chebyshev coefficients $c_{cgt} \in \mathbb{R}^K$. Truncated Chebyshev series closely approximate the minimax polynomial, i.e. $c_{cgt} \approx \operatorname{arg min}_c \| c_{gt} - \sum_k c_k T_k \|_\infty$ where $T_k$ is the Chebyshev polynomial of order $k$. Given that the polynomials form an orthogonal basis for $L^2([-1,1],\frac{dy}{\sqrt{1-y^r}})$, the coefficients can be retrieved by two methods.

1. Analytical projection.
    * $c_k = \frac{2}{\pi} \int_0^\pi \cos(k\theta) g( \frac{\lambda_{max}}{2} (\cos(\theta) + 1)) d\theta$
    * Need the analytic function.
2. Numerical projection (discrete orthogonality condition).
    * $c_k = \frac{2}{K} \sum_j g(x_j) T_k(x_j)$ where the $x_j$ are the $K$ Chebyshev nodes, because the approximation error is null only at these points.
    * Need function evaluations at the Chebyshev nodes, but those only. Much less points than least mean square.

In our setting, the generative filter is the function to learn. We have however access to some evaluations of the filter (at the eigenvalues of the Laplacian) via convex optimization of the loss function $L$ (described above). From those, given the Chebyshev basis, we can retrieve the coefficients that minimize the reconstruction error of this filter.

Results:

* Playing with the order $K$ shows that the approximation converges to the filter $g$.
* The approximation constructed by minimizing the filter l2 reconstruction error is now longer a Chebyshev polynomial (there are error on the Chebyshev nodes) but it provides a smaller loss $L$ (our final measure of quality). It however requires the full Chebyshev basis, which requires the eigenvalues of the Laplacian.

In [ ]:
K = 10

def rescale(x, reverse=False):
    """Rescale the spectral domain to [-1,1]."""
    if normalized_laplacian:
        lmax = 2
    if reverse:
        return x / lmax * 2 - 1
    else:
        return (x + 1) / 2 * lmax
np.testing.assert_allclose(lamb, rescale(rescale(lamb, True)), atol=tol)

def cheby_nodes(K):
    """Return the K Chebyshev nodes in [-1,1]."""
    return np.cos(np.pi * (np.arange(K) + 1/2) / K)
    
def cheby_coeff(K, f):
    """Compute the coefficients of the Chebyshev polynomial approximation."""
    # Coefficients from discrete orthogonality condition.
    # It can be done faster via the discrete cosine transform.
    c = np.empty(K)
    x = cheby_nodes(K)
    T = cheby_basis(K, x)
    for k in range(K):
        c[k] = 2 / K * np.sum(f(x) * T[:,k])
    c[0] /= 2
    return c

# Domain is [-1, 1].
x = np.linspace(-1,1,100)
x = rescale(lamb, True)
f = lambda x: g(rescale(x))
np.testing.assert_allclose(f(x), c_g)

c_cg = cheby_coeff(K, f)
np.testing.assert_allclose(f(cheby_nodes(K)), cheby_basis(K, cheby_nodes(K)) @ c_cg)

T = cheby_basis(K, x)
c_co = np.linalg.lstsq(T, c_g)[0]

plot_filters(['T @ c_co', 'T @ c_cg', 'c_g'])
plt.plot(rescale(cheby_nodes(K)), f(cheby_nodes(K)), 'k.', markersize=15, label='Chebyshev nodes');

### Polynomial order

Determine the polynomial order by filtering the data with Chebyshev approximations of order $1 \leq k \leq K$ and monitoring the reconstruction loss $L$.

* The result shows that the approximation does indeed converge.
* The approximation loss arrives at a plateau (the round-off error ?) given a high enough order.
* As anticipated on the figure above, the coefficients provided by least square reconstruction have smaller loss than the *correct* ones.

In [ ]:
def polynomial_order(K):
    loss_cg = np.empty((K))
    loss_co = np.empty((K))
    for k in range(1, K+1):
        T = cheby_basis(k, x)
        c_cg = cheby_coeff(k, f)
        loss_cg[k-1] = L(T @ c_cg)
        c_co = np.linalg.lstsq(T, f(x))[0]
        loss_co[k-1] = L(T @ c_co)
    fig = plt.figure(figsize=(15,5))
    ax = fig.add_subplot(1,1,1)
    ax.semilogy(range(1,K+1), loss_cg, label='L(T @ c_cg)')
    ax.semilogy(range(1,K+1), loss_co, label='L(T @ c_co)')
    ax.semilogy(L(c_o) * np.ones(K+1), label='L(c_o)')
    ax.set_title('Loss due to Chebyshev approximation')
    ax.set_xlabel('Polynomial order')
    ax.set_ylabel('Loss L')
    ax.set_xlim(1, K)
    ax.legend(loc='best')
    plt.show()
polynomial_order(60)

Choose the polynomial order $K$ and compute the basis $T$ with their associate coefficients $c_{cgt}$.

In [ ]:
K = 15
t_start = time.process_time()
c_cg = cheby_coeff(K, f)
T = cheby_basis(K, x)
print('Execution time: {:1.0f}ms'.format((time.process_time() - t_start) * 1000))

# If the order is sufficient for a perfect (as good as c_gt) reconstruction (test only).
pol_order_is_sufficient = False

### Loss function

* Independant coefficients: $L = \frac{1}{N} \sum_{i=1}^M \| (Tc)_i (U^T X)_{i,\cdot} - (U^T Y)_{i,\cdot} \|_2^2$.
* $L = \frac{1}{N} \| Tc \circ U^T X - U^T Y \|_2^2$.
* $\nabla_{c} L = \frac{2}{N} \left(T^T \left( U^T X \circ ( Tc \circ U^T X - U^T Y ) \right) \right) 1_N$.

In [ ]:
def filter_chebyshev(X, c):
    """Filter X with the Chebyshev coefficients of the full filter."""
    return filter_full(X, T @ c)
c = np.zeros(K)
c[0] = 1
np.testing.assert_allclose(filter_chebyshev(X, c), X, atol=tol)

def Lc(c):
    M, N = X.shape
    return np.linalg.norm(filter_chebyshev(X, c) - Y, ord='fro')**2 / N
np.testing.assert_allclose(Lc(c_cg), L(T @ c_cg), atol=tol)
if pol_order_is_sufficient:
    np.testing.assert_allclose(Lc(c_cg), M * eps**2, rtol=1e-2, atol=tol)
    np.testing.assert_allclose(Lc(c_cg), L(c_g), atol=tol)

def dLc(X, Y, c):
    M, N = X.shape
    A = U.T @ X
    B = U.T @ Y
    return 2 / N * T.T @ (A * ((T @ c)[:,np.newaxis] * A - B)).sum(axis=1)
# Gradient should be null at the global minimum. With noise, c_cg is not necessary the optimum.
if eps <= 0 and pol_order_is_sufficient:
    np.testing.assert_allclose(dLc(X, Y, c_cg), 0, atol=tol)

### Optimality condition

* Given the signals $X$, $Y$ and the Chebyshev basis $T$, find the Chebyshev coefficients $c_{copt}$.
* Optimality condition $\nabla_c L = 0$ gives $(U^T X \circ U^T X) 1_N \circ Tc = (U^T X \circ U^T Y) 1_N$.
* Why do we not always reach the minimum, i.e. $\nabla_c L = 0$ ? E.g. for small polynomial orders, when we are not able to sufficiently approximate the filter.

In [ ]:
t_start = time.process_time()
c_co = np.linalg.lstsq(T, c_o)[0]
print('Execution time: {:1.0f}ms'.format((time.process_time() - t_start) * 1000))

assert Lc(c_co) < Lc(c_cg) + tol
assert np.linalg.norm(dLc(X, Y, c_co)) < np.linalg.norm(dLc(X, Y, c_cg))
#np.testing.assert_allclose(dLc(X, Y, c_co), 0, atol=tol)
if eps <= 0 and pol_order_is_sufficient:
    np.testing.assert_allclose(Lc(c_co), Lc(c_cg), atol=tol)

### Stochastic gradient descent

* Why |dL(c)| does not converge to the null vector ? There should be no gradient at the optimum.
* Convergence seems harder than before.

In [ ]:
c0 = np.random.uniform(0, 1, K)
c_cs = sgd_plot_convergence(c0, Lc, dLc, [[0.005, 100]], conv=1e-3, maxit=100)[0]

### Results: learned filters

* The coefficients `c_co`, being optimal, alwas have a smallest loss than the ground truth `c_cg` (interpolant at the Chebyshev points).
* The SGD solution `c_cs` does not converge exactly to `c_co`.

In [ ]:
def plot_coefficients(coeffs):
    fig = plt.figure(figsize=(15,5))
    ax = fig.add_subplot(1,1,1)
    for coeff in coeffs:
        c = eval(coeff)
        label = '{}: L={:1.2e}, |dL|={:1.2e}'.format(coeff, Lc(c), np.linalg.norm(dLc(X,Y,c)))
        ax.plot(c, 'o', label=label)
    ax.set_xlim(-1, K)
    ax.set_title('Chebyshev expansion coefficients, M={}, N={}, K={}, eps={}'.format(M, N, K, eps))
    ax.set_xlabel('number')
    ax.set_ylabel('value')
    ax.legend(loc='best')

plot_coefficients(['c_cs', 'c_co', 'c_cg'])
plot_filters(['T @ c_cs', 'T @ c_co', 'T @ c_cg', 'c_o'])

## Filter learning: recursive Chebyshev expansion

* Compute the coefficients via the recursive generation formula of the Chebyshev polynomials.
* Goals:
    * Avoid the $O(M^2)$ filtering operation, i.e. the multiplication with the Fourier basis to transform the signal in the spectral domain (convolution with the filter in the spatial domain).
        * Avoid the expensive EVD of the large (but sparse) Laplacian matrix needed to retrieve the Fourier basis. 
    * Avoid the computation of the Chebyshev basis.

Steps:

* Estimate the upper bound $\lambda_{max}$ of the Laplacian spectrum, i.e. an upper bound on the largest eigenvalue $\lambda_{M-1}$.
    * Normalized Laplacian spectrum is upper bounded by 2.
* We will need the adjoint for back-propagation.

Overall complexity from $O(M^2 \max(M,N))$ to $O(KMN)$.

### Chebyshev recursive formula

Step 1: Given the Chebyshev coefficients $c$, recursively refine $X$ into $\hat{Y}$. Our goal is to avoid the expensive $O(M^2)$ multiplication by the Fourier basis $U$.

* Compute the projection of the signal on the Chebyshev basis: $\hat{Y}_k = T_k X$. Keeping this result allows fast application of multiple set of coefficients.
    * The trick here is that each $\hat{Y}_k$ is recursively generated from $\hat{Y}_{k-1}$ and $\hat{Y}_{k-2}$ with only a multiplication by the sparse Laplacian matrix $\mathcal{L}$.
    * This operation thus costs $O(K |E| N) = O(KMN)$ operations if the number of edges $|E|$ is proportional to the number of nodes $M$ (e.g. for kNN graphs) while transforming the signals into the Fourier domain costs $O(M^2N)$.
    * This is done once for every signal. The filter coefficients $c$ can then be applied multiple times during learning.
    * Memory: $\{\hat{Y}_k\} \in \mathbb{R}^{K \times M \times N}$.
* The filtered signal is then given by $\hat{Y} = \sum_k c_k \hat{Y}_k$
    * This operation requires $O(KMN)$ operations where filtering in the Fourier domain costs $O(MN)$. But there will be many of these operations during SGD.

In [ ]:
def rescale_L(L):
    """Rescale the Laplacian eigenvalues in [-1,1]."""
    M, M = L.shape
    I = scipy.sparse.identity(M, format='csr')
    if normalized_laplacian:
        return L - I
    else:
        return L / lmax * 2 - I
    
def cheby_basis_eval(L, X, K):
    """Return T_k X where T_k are the Chebyshev polynomials of order up to K.
    Complexity is O(KMN)."""
    M, N = X.shape
    L = rescale_L(L)
    # Y = T @ X: MxM @ MxN.
    Y = np.empty((K, M, N))
    # Y_0 = T_0 X = I X = X.
    Y[0,...] = X
    # Y_1 = T_1 X = L X.
    if K > 1:
        Y[1,...] = L.dot(X)
    # Y_k = 2 L Y_k-1 - Y_k-2.
    for k in range(2, K):
        Y[k,...] = 2 * L.dot(Y[k-1,...]) - Y[k-2,...]
    return Y
np.testing.assert_allclose(cheby_basis_eval(LL, X, 1)[0,...], X)

def filter_recursive(Yhat, c):
    K, M, N = Yhat.shape
    Yhat.shape = (K, M*N)
    Y = c @ Yhat
    Y.shape = (M, N)
    return Y

### Clenshaw's method

* An algorithm which can evaluate any 3-relations recusive polynomial given the coefficients.
* The complexity of filtering with Clenshaw is $O(KMN)$, exactly the same as the standard Chebyshev recursion.
* The advantage of Chebyshev recursion is the possibility to store the $\hat{Y}_k$, which diminishes the computation cost by two for successive evaluation with different coefficients.
* Is the approximation error of this method smaller ? Otherwise I don't see why numerical packages like scipy uses it.

In [ ]:
def eval_clenshaw(x, c):
    K = len(c)
    b2 = 0
    b1 = c[K-1] * np.ones(x.shape) if K >= 2 else 0
    for k in range(K-2, 0, -1):
        b = c[k] + 2 * x * b1 - b2
        b2, b1 = b1, b
    return c[0] + x * b1 - b2

def test_basis(K, N=100):
    x = np.linspace(-1, 1, N)
    T = np.empty((N, K))
    for k in range(K):
        c = np.zeros(k+1)
        c[k] = 1
        T[:,k] = eval_clenshaw(x, c)
    np.testing.assert_allclose(T, cheby_basis(K, x))
test_basis(50)

def filter_clenshaw(L, X, c):
    K = len(c)
    L = rescale_L(L)
    B2 = 0
    B1 = c[K-1] * X if K >= 2 else np.zeros(X.shape)
    for k in range(K-2, 0, -1):
        B = c[k] * X + 2 * L.dot(B1) - B2
        B2, B1 = B1, B
    return c[0] * X + L.dot(B1) - B2

### Testing polynomials evaluation and filtering

* The first method seems faster because the Laplacian is already diagonalized.
* Clenshaw seems faster because it doesn't keep the intermediate result.
* Execution times for M=1000, N=200, K=15: 1617ms 36ms 44ms.

In [ ]:
def test_compare(c):
    t_start = time.process_time()
    T = cheby_basis(len(c), x)
    Y1 = filter_full(X, T @ c)
    t_full = (time.process_time() - t_start) * 1000
    
    t_start = time.process_time()
    Yhat = cheby_basis_eval(LL, X, len(c))
    Y2 = filter_recursive(Yhat, c)
    t_cheby = (time.process_time() - t_start) * 1000
    np.testing.assert_allclose(Y1, Y2, atol=tol)
    
    t_start = time.process_time()
    Y2 = filter_clenshaw(LL, X, c)
    t_clenshaw = (time.process_time() - t_start) * 1000
    np.testing.assert_allclose(Y1, Y2, atol=tol)
    
    print('Execution times: {:1.0f}ms {:1.0f}ms {:1.0f}ms'.format(t_full, t_cheby, t_clenshaw))

test_compare(np.array([1]))
test_compare(np.array([1,0,0,0]))
test_compare(np.array([0,1,0,0]))
test_compare(np.array([0,0,1,0]))
test_compare(np.array([0,0,0,1]))
test_compare(np.random.uniform(0, 5, size=100))
test_compare(c_cg)
test_compare(c_co)

### Loss function and optimality condition

Minimize $L = \frac{1}{N} \|\hat{Y} - Y\|_F^2$ where $\hat{Y} = \sum_k c_k \hat{Y}_k$.

* Rewrite as $L = \frac{1}{N} \|\bar{Y} c - \bar{y}\|_F^2$ where $\bar{y}$ is the vectorized matrix $Y$ and the $k^\text{th}$ column of $\bar{Y}$ is the vectorized matrix $\hat{Y}_k$.
* Gradient $\nabla_c L = \frac{2}{N} \bar{Y}^T (\bar{Y} c - \bar{y})$
* Optimality condition $\bar{Y} c = \bar{y}$
    * Largely over-determined as $K << MN$; $\bar{y} \in \mathbb{R}^{NM}$ and $c \in \mathbb{R}^K$.

1. Compute these coefficients without the basis, thus without the eigenvalues.
    * And we avoid the $O(M^3)$ EVD.
    * Avoid the EVD.

In [ ]:
def vectorize(Yhat, Y):
    K, M, N = Yhat.shape
    return Yhat.reshape((K, M*N)), Y.reshape((M*N))

def Lcr(c):
    Yhat = cheby_basis_eval(LL, X, len(c))
    return np.linalg.norm(filter_recursive(Yhat, c) - Y, ord='fro')**2 / N

def dLcr(X, Y, c):
    Yhat = cheby_basis_eval(LL, X, len(c))
    Ybar, ybar = vectorize(Yhat, Y)
    return 2 / N * (c @ Ybar - ybar) @ Ybar.T

def cheby_coeff_opt(X, Y, K):
    Yhat = cheby_basis_eval(LL, X, len(c))
    Ybar, ybar = vectorize(Yhat, Y)
    return np.linalg.lstsq(Ybar.T, ybar)[0]

t_start = time.process_time()
c_cro = cheby_coeff_opt(X, Y, K)
print('Execution time: {:1.0f}ms'.format((time.process_time() - t_start) * 1000))

np.testing.assert_allclose(Lcr(c_cro), L(T @ c_cro), atol=tol)
assert Lcr(c_cro) < Lcr(c_cg) + tol
assert Lcr(c_cro) < Lcr(c_co) + tol
if pol_order_is_sufficient:
    np.testing.assert_allclose(Lcr(c_cro), M * eps**2, rtol=2e-2, atol=tol)
if eps <= 0 and pol_order_is_sufficient:
    np.testing.assert_allclose(Lcr(c_cro), Lcr(c_co), atol=tol)
np.testing.assert_allclose(dLcr(X, Y, c_cro), 0, atol=tol)
assert np.linalg.norm(dLcr(X, Y, c_cro)) < np.linalg.norm(dLcr(X, Y, c_cg)) + tol
assert np.linalg.norm(dLcr(X, Y, c_cro)) < np.linalg.norm(dLcr(X, Y, c_co)) + tol

### Stochastic gradient descent

In [ ]:
c0 = np.random.uniform(0, 1, K)
c_crs = sgd_plot_convergence(c0, Lcr, dLcr, [[.01, 100]], conv=1e-3, maxit=100)[0]

### Results: learned filters

* Fast decay of the coefficients: good for approximation.
* The *ground truth* and *optimal* coefficients are similar, given a sufficiently high order $K$. Otherwize the *optimal* are better.


* The optimal solutions `c_co` and `c_cro` are close and have the smallest loss.
* The SGD solutions `c_cs` and `c_crs` are close while a bit less accurate.
    * Probably the convergence isn't that great.
    * They approximate the hight frequencies the least accurately.
* The ground truth solution lies in the middle.

In [ ]:
plot_coefficients(['c_crs', 'c_cro', 'c_cs', 'c_co', 'c_cg'])
plot_filters(['T @ c_crs', 'T @ c_cro', 'c_o'])